# DSC syntax: basics of benchmark execution

This document is continuation of discussion in [DSC syntax: basics of modules](DSC_Configuration.html). We will use toy DSC examples (including breaking down [this toy](https://github.com/stephenslab/dsc2/blob/master/vignettes/one_sample_location/settings.dsc)) to introduce a special syntax block called `DSC` that defines benchmark pipelines and execution. This block has a reserved headline `DSC` and is required for all DSC scripts. In this document we refer to its contents by `DSC::<key>` where `<key>` is the identifier on the left side of `:` in each line of the `DSC` block.

## Pipeline operators

To put together modules into pipelines, two logic are available: "alternating" modules refers to groups of modules that achieve the same goal (able to take similar input and provide the same type of output) as if they are exchangable; "concatenating" modules refers to groups of modules that acts one after another as if concatenated into a chain of actions. These logic are implemented via `*`, `,` and `()` operators:

*  `*`: connects concatenating modules, ie, right hand side (RHS) is executed after the left hand side (LHS).
*  `,`: connects alternating modules, ie, RHS and LHS are exchangable.
*  `()`: when alternating modules and concatenating modules are combined, it is used to re-define operator precedence (by default `*` has higher precedence than `,`).

## Module ensembles and pipelines

`DSC::define` defines module ensembles, that is, groups of "alternating" or "concatenating" modules. For example:

```
normal, t: rnorm.R, rt.R
    ...
mean, median: mean.R, median.R
    ...

DSC:
    define:
      simulate: normal, t
      estimate: mean, median
```

Here 2 ensembles are defined: `simulate`, of alternating modules `normal` and `t` for data generation, and `estimate`, of alternating modules `mean` and `median` for parameter estimation.

To illustrate concatenating modules and its combination with alternating modules we can add an additional `winsorize` module to `estimate`:

```
DSC:
    define:
      simulate: normal, t
      winsorized_estimate: winsorize * (mean, median)
```

These ensembles, once defined, can be used as if they are modules, to define DSC benchmark as will be discussed next.

## DSC benchmark

DSC benchmark, defined by `DSC::run`, is one or multiple DSC pipelines each consisted of logically connected modules or module ensembles via pipeline operators. We will use a series of examples to illustrate how benchmarks are created.

#### Example 1: ensembles in pipeline
```
run: simulate * estimate * score
```

Here `simulate` and `estimate` are module ensembles (see `DSC::define` of previous section). This is equivalent to writing:

```
run: (normal, t) * (mean, median) * score    
```

#### Example 2: combination of concatenating and alternating modules

```
run: simulate * (winsorize * mean, mean) * score
```
DSC will run 2 types of pipelines: 1) that involves `winsorize` followed by `mean`, and 2) "mean" only.

#### Example 3: expansion of pipeline operators

```
run: simulate * (voom, sqrt, identity) * (RUV, SVA) * (DESeq, edgeRglm, ash) * score
```

will be expanded to:

```
run: simulate * sqrt * RUV * DESeq * score,
   simulate * sqrt * SVA * DESeq * score,
   simulate * identity * RUV * DESeq * score,
   simulate * voom * RUV * DESeq * score,
   simulate * identity * SVA * DESeq * score,
   simulate * voom * SVA * DESeq * score,
   simulate * sqrt * RUV * ash * score,
   simulate * sqrt * RUV * edgeRglm * score,
   simulate * sqrt * SVA * ash * score,
   simulate * sqrt * SVA * edgeRglm * score,
   simulate * identity * RUV * ash * score,
   simulate * voom * RUV * ash * score,
   simulate * identity * RUV * edgeRglm * score,
   simulate * voom * RUV * edgeRglm * score,
   simulate * identity * SVA * ash * score,
   simulate * voom * SVA * ash * score,
   simulate * identity * SVA * edgeRglm * score,
   simulate * voom * SVA * edgeRglm * score
```

## Command interface to DSC benchmark
By default, DSC will execute all pipelines defined in `DSC::run`. But one can overwrite sequence from [command line](Command_Options.html). For example to execute the complete `Example 1`

```
dsc file.dsc ... --target "simulate * estimate * score"
```
or a subset of `Example 1`:

```
dsc file.dsc ... --target "norm * estimate * score"
```

You can also execute a single module / ensemble **for debug purpose**:

```
dsc file.dsc ... --target "simulate"
```

## Benchmark parameters

These parameters are specified in the rest of `DSC` block:

* `DSC::exec_path`: search path for executable files.
* `DSC::lib_path`: search path for library files, such as R script to be `source`-ed or Python scripts to be `import`-ed.
* `DSC::R_libs`: check required R libraries and if not available, automatically install them from cran, bioconductor or github. 3 formats are accepted:
 * `<package>`: DSC will check / install `package` from cran or bioconductor.
 * `<github_repo/package>`: DSC will install `package` from `github_repo` on github.
 * `<package (version[s])>`: DSC will install `package` and check version as required. A warning message will be generated if version do not match. It is possible to specify several versions such as `edgeR (3.12.0, 3.12.1)` or simply `edgeR (3.12.0+)`.
* `python_modules`: check required Python3 modules, and install from pip if not available.
* `work_dir`: work directory where benchmarking will take place.
* `output`: folder name to store benchmark output files.
* `global`: defines global variables which can be used in definition of modules through [substitution operator `$()`](DSC_Configuration.html#Substitution-operator-131).

## Ready to indulge?
At this point we have covered the basics to compose a DSC benchmark. To see how it works in practice with a simple example, please follow through this [Quick Start tutorial](tutorials/Quick_Start.html).